 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/ch131-Database-Access/sqlite.ipynb#Chapter-131:-Database-Access" data-toc-modified-id="Chapter-131:-Database-Access-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Chapter 131: Database Access</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/ch131-Database-Access/sqlite.ipynb#Section-131.1:-SQLite" data-toc-modified-id="Section-131.1:-SQLite-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Section 131.1: SQLite</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/ch131-Database-Access/sqlite.ipynb#對-cursor-執行-SQL-語法並透過-connection-commit/save-結果" data-toc-modified-id="對-cursor-執行-SQL-語法並透過-connection-commit/save-結果-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>對 cursor 執行 SQL 語法並透過 connection commit/save 結果</a></span></li><li><span><a href="http://localhost:8888/notebooks/ch131-Database-Access/sqlite.ipynb#用-cursor-查看剛才-commit-的-execute-結果" data-toc-modified-id="用-cursor-查看剛才-commit-的-execute-結果-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>用 cursor 查看剛才 commit 的 execute 結果</a></span></li><li><span><a href="http://localhost:8888/notebooks/ch131-Database-Access/sqlite.ipynb#使用-in-memory-sqlite" data-toc-modified-id="使用-in-memory-sqlite-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>使用 in-memory sqlite</a></span></li><li><span><a href="http://localhost:8888/notebooks/ch131-Database-Access/sqlite.ipynb#把上一次-commit-之後的所有-execute-紀錄消去重來" data-toc-modified-id="把上一次-commit-之後的所有-execute-紀錄消去重來-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>把上一次 commit 之後的所有 execute 紀錄消去重來</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/ch131-Database-Access/sqlite.ipynb#Connection" data-toc-modified-id="Connection-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Connection</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/ch131-Database-Access/sqlite.ipynb#row_factory" data-toc-modified-id="row_factory-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span><code>row_factory</code></a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/ch131-Database-Access/sqlite.ipynb#Important-Functions-of-Cursor" data-toc-modified-id="Important-Functions-of-Cursor-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Important Functions of Cursor</a></span></li></ul></li></ul></div>

# Chapter 131: Database Access

## Section 131.1: SQLite

SQlite 不需要一個額外的 database server, 常用的 use cases 是 prototyping 或者是建立專給一個人使用的 db

In [1]:
import sqlite3

In [2]:
sqlite3.version

'2.6.0'

在目前的 folder 底下產生一個 file 叫做 `users.db` 來存放 db 資料並建立一個 connection  
要使用 in-memory db 可以用
```
conn = sqlite3.connect(':memory:')
```

In [5]:
conn = sqlite3.connect("users.db")

In [23]:
!ls

sqlite.ipynb users.db


In [6]:
c = conn.cursor()

### 對 cursor 執行 SQL 語法並透過 connection commit/save 結果

In [8]:
c.execute("CREATE TABLE user (name text, age integer)")

In [11]:
c.execute("INSERT INTO user VALUES ('User A', 42)")
c.execute("INSERT INTO user VALUES ('User B', 25)")

In [14]:
conn.commit()

### 用 cursor 查看剛才 commit 的 execute 結果

In [15]:
c.execute("SELECT * FROM user")

In [17]:
help(c.fetchall)

Help on built-in function fetchall:

fetchall(...) method of sqlite3.Cursor instance
    Fetches all rows from the resultset.



In [18]:
c.fetchall()

[('User A', 42), ('User B', 25)]

In [19]:
conn.close()

### 使用 in-memory sqlite

In [73]:
conn = sqlite3.connect(':memory:')

In [74]:
c = conn.cursor()

In [75]:
c.execute('''CREATE TABLE friends
            (id INTEGER, name TEXT, birthday DATE)''')
c.execute('''INSERT INTO friends
             VALUES (1, "lee", "1990-10-22")''')
conn.commit()

In [76]:
c.execute('''SELECT * FROM friends''')
c.fetchall()

[(1, 'lee', '1990-10-22')]

### 把上一次 commit 之後的所有 execute 紀錄消去重來

In [77]:
c.execute('''INSERT INTO friends
             VALUES (2, "lee-will-got-rollbacked-and-never-show", "1990-11-22")''')

In [78]:
conn.rollback()
conn.commit()

In [79]:
c.execute('''SELECT * FROM friends''')
c.fetchall()

[(1, 'lee', '1990-10-22')]

In [80]:
conn.close()

## Connection
- `isolation_level` 沒有特別設定的話, 沒有 `commit` 的 SQL execute 都會被自動捨棄
- 省略 `cursor`: 基本上可以不另外呼叫`cursor`直接用conn做SQL操作, 直接呼叫`execute()`, sqlite 就會自動幫我們建立一個`cursor` 物件並呼叫對應的 methods

In [123]:
conn = sqlite3.connect("users.db")

In [124]:
conn.isolation_level

''

In [125]:
conn.execute("INSERT INTO user VALUES ('User C will not be inserted!', 250)")
conn.close() # no commit before close!

In [126]:
conn = sqlite3.connect("users.db")

In [127]:
print(conn.execute("SELECT * FROM user").fetchall())
conn.close()

[('User A', 42), ('User B', 25)]


### `row_factory`
sqlite 預設每一 row 是一個 tuple, 利用`row_factory`可以自訂每一 row 的 repr, 像是`dict`

In [132]:
def dict_factory(cursor, row): 
    d = {}
    print('Row')
    for i, col in enumerate(cursor.description): 
        print('\ti: {}, col: {}'.format(i, col))
        d[col[0]] = row[i]
    return d
conn = sqlite3.connect("users.db")
conn.row_factory = dict_factory

In [133]:
conn.execute('SELECT * FROM user').fetchall()

Row
	i: 0, col: ('name', None, None, None, None, None, None)
	i: 1, col: ('age', None, None, None, None, None, None)
Row
	i: 0, col: ('name', None, None, None, None, None, None)
	i: 1, col: ('age', None, None, None, None, None, None)


[{'age': 42, 'name': 'User A'}, {'age': 25, 'name': 'User B'}]

In [134]:
conn.close()